In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, save_img
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layersimport Dense, GlobalAveragePooling2D
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.transform import resize
import zipfile
import shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Install and configure Kaggle
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

# Extract the dataset
with zipfile.ZipFile('/content/skin-cancer-mnist-ham10000.zip', 'r') as zip_ref:
zip_ref.extractall('dataset')

In [ ]:
# Load metadata
df = pd.read_csv("/content/dataset/HAM10000_metadata.csv")
print(df)
print(df['dx'].value_counts())
df.drop(['lesion_id', 'dx_type', 'sex', 'localization', 'age'], axis=1, inplace=True)
df = df.rename(columns={'dx': 'label'})
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
df['label'] = df['label'].astype(str)
print(df['label'].value_counts())

# Copy images to a single directory
source_folder1 = '/content/dataset/HAM10000_images_part_1'
source_folder2 = '/content/dataset/HAM10000_images_part_2'
destination_folder = '/content/images'
shutil.copytree(source_folder1, destination_folder)
shutil.copytree(source_folder2, destination_folder, dirs_exist_ok=True)

# Define image paths
df['image_path'] = df['image_id'].apply(lambda x: f'/content/images/{x}.jpg')
df['preprocessed_path'] = df['image_id'].apply(lambda x: f'/content/preprocessed_images/{x}.jpg')

os.makedirs('/content/preprocessed_images', exist_ok=True)

def preprocess_and_save_image(row):
    img = load_img(row['image_path'])
    img = img_to_array(img)
    img = resize(img, (224, 224), preserve_range=True, anti_aliasing=True)

    # Apply Non-Local Means filter
    sigma_est = np.mean(estimate_sigma(img, multichannel=True))
    patch_kw = dict(patch_size=5, patch_distance=6, multichannel=True)
    img = denoise_nl_means(img, h=1.15 * sigma_est, fast_mode=True, **patch_kw)

    img = preprocess_input(img)
    save_img(row['preprocessed_path'], img)

    df.apply(preprocess_and_save_image, axis=1)

In [ ]:
# Display images before and after preprocessing
img1 = mpimg.imread('/content/images/ISIC_0024306.jpg')
img2 = mpimg.imread('/content/preprocessed_images/ISIC_0024306.jpg')
plt.imshow(img1)
plt.axis('off')
plt.show()
plt.imshow(img2)
plt.axis('off')
plt.show()

In [ ]:
img1 = mpimg.imread('/content/images/ISIC_0024309.jpg')
img2 = mpimg.imread('/content/preprocessed_images/ISIC_0024309.jpg')
plt.imshow(img1)
plt.axis('off')
plt.show()
plt.imshow(img2)
plt.axis('off')
plt.show()

In [ ]:
# Verify the filenames are correctly created
df['filename'] = df['image_id'].apply(lambda x: f'{x}.jpg')
print(df.head())    # Check the dataframe to ensure 'filename' column is correctly created

# Data generator
image_dir = '/content/preprocessed_images'
datagen = ImageDataGenerator(validation_split=0.2)


train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=image_dir,
    x_col='filename'
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)    

validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=image_dir,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='validation'

)

# Load MobileNet base model without the top layer, suited for your input size

base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Assemble the model for multiclass classification
n_classes = 7
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(n_classes, activation='softmax')
])

# Compile the model for multiclass classification
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10
)

# Save the model
model.save('/content/mobilenet.h5')

In [ ]:
# Evaluate the model
train_loss, train_accuracy = model.evaluate(train_generator, steps=train_generator.samples // train_generator.batch_size)
print(f"Loss: {train_loss}, Accuracy: {train_accuracy}")

In [ ]:
# Make a prediction on a sample image
image_path = '/content/preprocessed_images/ISIC_0024309.jpg'
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array_expanded = np.expand_dims(img_array, axis=0)
img_preprocessed = preprocess_input(img_array_expanded)

# Load the trained Keras model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(n_classes, activation='softmax')

]) 
model.load_weights ('/content/mobilenet.h5')

# Predict the class of the image
predictions = model.predict(img_preprocessed)
predicted_class_index = np.argmax(predictions[0])
predicted_probability = predictions[0][predicted_class_index]

# Display the image and prediction
plt.imshow(img)
plt.axis('off')
plt.show()

print(f"Model's Prediction: Class {predicted_class_index} (Probability: {predicted_probability})")

In [ ]:
import gradio as gr

# Function to predict and display the image and probability
def predict(image_name):
# Define the mapping of class indices to cancer type names
    class_names = {
        0: 'Actinic Keratoses',
        1: 'Basal Cell Carcinoma',
        2: 'Benign Keratosis-like Lesions',
        3: 'Dermatofibroma',
        4: 'Melanocytic Nevi's,
        5: 'Melanoma',
        6: 'Vascular Lesions'

    }    

    image_path = f'/content/preprocessed_images/{image_name}'
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    img_preprocessed = preprocess_input(img_array_expanded)

    # Predict the class of the image
    predictions = model.predict(img_preprocessed)
    predicted_class_index = np.argmax(predictions[0])
    predicted_probability = predictions[0][predicted_class_index]

    # Get the cancer type name
    predicted_cancer_type = class_names[predicted_class_index]

    # Return the prediction and the image
    return (f"Model's Prediction: {predicted_cancer_type} (Probability: {predicted_probability})", image_path)

# Gradio UI 
interface = gr.Interface(
    fn=predict,


    

